In [1]:
import pandas as pd
import numpy as np

In [2]:
##Read in csv 
met_df = pd.read_csv(r'C:\Users\Leanne\Desktop\ELT_data\MetObjects.csv')

C:\Users\Leanne\Anaconda2\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
#Create a list of the columns we want to pull, based on the db schema created
columns = ['Department','Object ID', 'Object Name', 'Title', 'Artist Alpha Sort', 'Artist Begin Date', 'Artist End Date', 'Object Begin Date', 'Object End Date', 'Medium', 'Classification', 'Link Resource']

#Create a dataframe that only contains 'painting' objects
painting_filter = met_df['Object Name'].isin(["Painting"]) 
paintings = pd.DataFrame(met_df[painting_filter], columns=columns)

#display counts to identify columns
#paintings.head(1)

In [27]:
#read out counts to identify columns with missing values - total is to have 5,887
paintings.count()

Department           5887
Object ID            5887
Object Name          5887
Title                5553
Artist Alpha Sort    5654
Artist Begin Date    5451
Artist End Date      5454
Object Begin Date    5887
Object End Date      5887
Medium               5887
Classification       5887
Link Resource        5887
dtype: int64

In [28]:
# 334 records are missing title, which we want to drop...so drop those. 
paintings = paintings.dropna(subset=['Title'])

In [29]:
#Some of the default entries for artist birth/date are standardized to '9999', which we'll want to replace with ours ('unknown')
#Then we discover that there are 'citation' credits that include multiple artists names and dates seperated by "|"
#We only want the first/primary artist, not the citation, so we'll split str and just dump the extra
paintings['Artist Alpha Sort'] = paintings['Artist Alpha Sort'].str.split('|', expand=True)
paintings['Artist Begin Date'] = paintings['Artist Begin Date'].str.split('|', expand=True)
paintings['Artist End Date'] = paintings['Artist End Date'].str.split('|', expand=True)

In [30]:
#We read out unique values for Artist Alpha Sort. There is A LOT of junk for artist. 
#We were hoping that the Alpha Sort variable instead of the general 'Artist Name' would be more standardized for searching/indexing, etc. 
#...It's not. But there are mono-name artists (Goya, Raphael) that we risk losing if we try to parse via a delimiter like a comma

In [31]:
#Additionally, there are some values where a nickname is in parenthesis & that we'll want to clean these too
#I couldn't find a good way to set a start and end delimiter for a split, so at this point it might be easier to break out into first and last name
#then split on that. 

#There were hidden characters, so we had to have two extra holding columns, then delete
paintings[['Artist_Surname', 'Artist_First', 'Blank1', 'Blank2']] = paintings['Artist Alpha Sort'].str.split(",",expand=True)
paintings = paintings.drop(columns=['Blank1', 'Blank2'])

#Drop anything after a "(" for both new columns, then remove any potential spaces
paintings['Artist_Surname'] = paintings['Artist_Surname'].str.split('(', expand=True)
paintings['Artist_First'] = paintings['Artist_First'].str.split('(', expand=True)

paintings['Artist_Surname'] = paintings['Artist_Surname'].str.strip()
paintings['Artist_First'] = paintings['Artist_First'].str.strip()

In [32]:
#Now create a new 'full name' to match to as a stepping-stone between artist_id and original source
#Note that the 'blank_' is to account for single-name artists, like Raphael, Goya, etc.
#I.e., replace Artist_clean with our new, cleaned out first + last. And then run a strip to remove any spaces, just in case
paintings['Artist_First'] = paintings['Artist_First'].fillna("blank_")
paintings['Artist_Full_Name'] = paintings['Artist_First'].str.cat(paintings['Artist_Surname'],sep=" ")
paintings['Artist_Full_Name'] = paintings['Artist_Full_Name'].replace('blank_',r'', regex=True)
paintings['Artist_Full_Name'] = paintings['Artist_Full_Name'].str.strip()

In [33]:
#Now replace all blanks, plus junk values of '9999' for End date
paintings['Artist End Date'] = paintings['Artist End Date'].str.replace('9999', 'Unknown')
paintings = paintings.fillna('Unknown')

In [34]:
#Now create a seperate artist table -- pull in full name (key), first name, surname, birth and death
#Drop duplicates and reset_index
artists = pd.DataFrame(paintings, columns=['Artist_Surname', 'Artist_First', 'Artist_Full_Name', 'Artist Begin Date', 'Artist End Date'])
artists = artists.drop_duplicates()

In [35]:
#Reset the index to get an incremental id, then save to a new columns ('Artist_id')
artists = artists.reset_index()
artists['Artist_id'] = artists.index
artists = artists.drop(columns=['index'])

In [37]:
#However, we still have to add the new artist index back to the paintings df, then remove any artist info (now duplicative)

#We'll create a working df that is just full name (key) and id. Then we'll merge on full name back into paintings
artists_id_merge = pd.DataFrame(artists, columns=['Artist_Full_Name', 'Artist_id'])
paintings_master = paintings.merge(artists_id_merge, on='Artist_Full_Name')

#Drop unnecessary artist info from paintings_master
paintings_master = paintings_master.drop(columns=['Artist Alpha Sort','Artist_Surname', 'Artist_First', 'Artist_Full_Name', 'Artist Begin Date', 'Artist End Date'])

In [38]:
#Now do the same thing, but with department
department = pd.DataFrame(paintings, columns=['Department'])
department = department.drop_duplicates()

department = department.reset_index()
department['Dept_id'] = department.index
department = department.drop(columns=['index'])

paintings_master = paintings_master.merge(department, on='Department')
paintings_master = paintings_master.drop(columns=['Department'])

In [46]:
#rename all columns in Paintings, Artists and Department to have underscores, so that they're all standardized
paintings_master = paintings_master.rename(columns={'Object ID':'Object_id', 'Object Name':'Object_type','Object Begin Date':'Date_started', 'Object End Date':'Date_finished','Link Resource':'Met_link'})
department = department.rename(columns={'Department':'Dept_name'})
artists = artists.rename(columns={'Artist_Surname':'Surname', 'Artist_First':'First_name', 'Artist_Full_Name':'Full_name', 'Artist Begin Date':'Birth_date', 'Artist End Date':'Death_date' })

In [47]:
#save out all tables to csv
paintings_master.to_csv('paintings_master.csv')
department.to_csv('department_table.csv')
artists.to_csv('artists_table.csv')